In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("../data/diamonds.csv")


# Hur ska datasetet hanteras för att få fram så pålitlig data som möjligt?
## Hantera x, y och z
- Efter att ha testat beräkning av x, y och z i förhållande till carat och depth har jag kommit fram till att det finns mycket felvärden i yttermåtten, och ska jag gå efter något så har jag carat och depth att utgå från, eftersom dessa bör vara mer korrekta än yttermåtten. Totalt var det ca 30 000 diamanter som utifrån inlagda x, y och z värden inte var korrekt beräknade i carat. Om man tar bort x, y och z försvinner också ett flertal extrema utliggare. Men ytterligare ett problem om man tar bort x, y och z är att många rader försvinner och jag vet ännu inte varför det är stor skillnad. 

## Lösning:
Efter städning skapar jag tre dataset:
- Behåll original-dataset: `df_clean_original`
    - Slå ihop x, y och z till ett värde som ger ett volymvärde istället.
- Skapa ett dataset med de korrekta beräkningarna: `df_clean_corr_carat`
    - Slå ihop x, y och z till ett värde som ger ett volymvärde istället.
- Skapa ett dataset med de inkorrekta beräkningarna: `df_clean_inc_carat`
    - Slå ihop x, y och z till ett värde som ger ett volymvärde istället.

Sedan jämför jag dessa för att se vilken som upplevs mest trovärdig.

# Att städa:
df_clean = df.copy()
## Ta bort nollvärden och extrema utliggare.
Kommer extremerna påverka mycket? Får också se det ur perspektivet att ska en Guldfyndsbutik sälja så stora diamanter? Troligen inte.

## Ändra object datatyp till category
För att kunna encode värdena till nummeriska värden
## Ta bort dubletter
För att få en lite mer tillförlitlig data
## Lägger till encoded för cut, color och clarity
Använder bara numeriska värden för kolumnerna
## Tar bort cut, color och clarity
För att få en mindre matris
## Dela upp carat i 10 lika stora categorier
För att det förmodligen finns en varians i korrelationen mellan carat och pris beroende på carat.
## Dela upp datasetet i tre: original, korrect carat baserad på x, y, z och inkorrekt carat
För att kunna sätta dataseten i relation till varandra för att se hur de förhåller sig till varandra. Skulle jag ha ett dataset för att visa helt korrekta carat i förhållande till volymen? Men åter igen blir det vilket värde är mest tillförlitligt.
## Ta bort x, y, z
För att x, y och z blir överflödigt eftersom jag har delat upp i tre dataset, som antingen har rätt beräknad carat och därmed rätt volymvärden, eller fel carat/yttermåt och då spelar dem ingen roll.
## Normalisera datan
För att kunna jämföra datan enklare. Sparar även en ny csv för varje dataset, så att jag kan återanvända det jag gjort när jag analyserar datan mot uppgiften.
Kom ihåg att använda:
```python
df_correct_carat_clean = pd.read_csv('data/normalized_diamonds_clean_corr_carat_without_x_y_z.csv', index_col=0)
df_incorrect_carat_clean = pd.read_csv('data/normalized_diamonds_clean_inc_carat_without_x_y_z.csv', index_col=0)
df_original_clean = pd.read_csv('data/normalized_diamonds_clean_without_x_y_z.csv', index_col=0)
```


# Cleaning

In [3]:
df_clean = df.copy()

## Ta bort nollvärden och extrema utliggare.

In [4]:
# Tar bort nollvärden
df_clean = df_clean[(df.x > 0) & (df.y > 0) & (df.z > 0)]

# Hantera outliers i numeriska kolumner
# Välja ut numeriska kolumner
columns_to_check = df_clean.select_dtypes(include=['number']).columns 

# Skapa en ny df för att spara utliggarna
outlier_info = pd.DataFrame(index=df_clean.index)

# Hitta utliggare
for col in columns_to_check:
    # Kvartiler
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    extreme_lower = Q1 -3.0 * IQR
    extreme_upper = Q3 + 3.0 * IQR

    # Lägger till en column med en markering om ett värde är utliggare
    outlier_info[f"{col}_outlier"] = ((df_clean[col] < lower) | (df_clean[col] > upper))
    # Lägger till en column med markering om värdet är extrem utliggare
    outlier_info[f"{col}_extreme"] = ((df_clean[col] < extreme_lower) | (df_clean[col] > extreme_upper))

    # Beräknar hur många utliggare som finns
    outliers_count = outlier_info[f"{col}_outlier"].sum()
    extreme_count = outlier_info[f"{col}_extreme"].sum()

    print(f"{col}: {outliers_count} outliers ({outliers_count/len(df_clean)*100:.1f}%)")
    print(f"{col}: {extreme_count} extreme ({extreme_count/len(df_clean)*100:.1f}%)")

# Skapa sammanfattningskolumner
outlier_info['total_outlier_flags'] = outlier_info.filter(like='_outlier').sum(axis=1)
outlier_info['total_extreme_flags'] = outlier_info.filter(like='_extreme').sum(axis=1)

# Slår ihop summering med städat df
df_clean_with_extremes = pd.concat([df_clean, outlier_info], axis=1)

# Skapar en df utan extremer
df_clean = df_clean[outlier_info['total_extreme_flags'] == 0].copy()
print(f"\nRemoved {len(df_clean_with_extremes) - len(df_clean)} extreme outlier rows")
print(f"Retained {len(df_clean)} clean rows ({len(df_clean)/len(df_clean_with_extremes)*100:.1f}%)")
print(df_clean)

carat: 1883 outliers (3.5%)
carat: 40 extreme (0.1%)
depth: 2543 outliers (4.7%)
depth: 278 extreme (0.5%)
table: 604 outliers (1.1%)
table: 28 extreme (0.1%)
price: 3532 outliers (6.6%)
price: 121 extreme (0.2%)
x: 24 outliers (0.0%)
x: 0 extreme (0.0%)
y: 22 outliers (0.0%)
y: 2 extreme (0.0%)
z: 29 outliers (0.1%)
z: 2 extreme (0.0%)

Removed 450 extreme outlier rows
Retained 53470 clean rows (99.2%)
       carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0   

## Ändra object datatyp till category

In [5]:
df_clean['cut'] = pd.Categorical(df_clean.cut, ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'], ordered=True)

df_clean['color'] = pd.Categorical(df_clean.color, ['J', 'I', 'H', 'G', 'F', 'E', 'D'], ordered=True)

# clarity är extra viktig för den är fel. Ska vara enligt: I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF
df_clean['clarity'] = pd.Categorical(df_clean.clarity, ["I1", "SI2", "SI1", "VS2", "VS1",
"VVS2", "VVS1", "IF"], ordered=True)

## Ta bort dubletter

In [6]:
duplicate_indices_full = []

duplicated_rows = df_clean.duplicated(keep='first')

for idx, is_duplicate in enumerate(duplicated_rows):
    if is_duplicate:
        duplicate_indices_full.append(idx)
        
print(f"Found {len(duplicate_indices_full)} duplicate rows")
print(f"Duplicate row indices: {duplicate_indices_full}")


df_clean = df_clean.drop(duplicate_indices_full, errors='ignore')
print(df_clean)


Found 142 duplicate rows
Duplicate row indices: [984, 985, 986, 987, 1998, 2155, 2783, 3116, 3663, 3940, 4341, 4448, 4473, 5120, 5605, 6274, 7641, 7673, 8255, 8522, 9116, 9512, 10093, 10276, 10383, 10917, 11149, 11221, 12166, 12193, 12542, 13753, 13892, 14807, 14896, 15201, 15339, 16147, 17009, 17422, 18544, 19527, 20475, 20890, 21444, 21622, 21984, 21986, 22213, 22230, 22642, 22684, 23158, 23561, 23805, 23985, 24679, 25153, 25754, 25880, 26219, 26354, 26408, 26823, 26843, 27426, 27742, 27778, 27986, 28002, 28254, 28690, 29169, 29252, 29318, 29463, 29823, 30051, 30172, 30236, 30327, 30544, 30911, 30982, 31285, 31287, 31908, 32309, 32690, 33037, 33079, 33463, 33719, 33814, 33835, 34077, 34185, 34317, 35102, 35222, 36098, 36130, 36197, 36302, 36514, 36822, 37022, 37142, 37143, 37387, 37639, 38028, 38145, 38211, 38274, 38366, 38503, 38559, 38694, 38765, 38871, 38995, 39077, 39146, 39728, 40094, 40750, 41300, 41422, 41731, 42597, 42873, 43036, 43110, 43180, 46163, 46242, 46657, 46881, 4754

## Lägger till encoded för cut, color och clarity

In [7]:
df_clean['cut_encoded'] = df_clean.cut.cat.codes
df_clean['color_encoded'] = df_clean.color.cat.codes
df_clean['clarity_encoded'] = df_clean.clarity.cat.codes

In [8]:
# Tar bort cut, color och clarity-kolumnerna
df_clean = df_clean.drop(['cut', 'color', 'clarity'], axis=1)


## Dela upp carat i 10 lika stora categorier


In [ ]:
df_clean['carat_bins'] = pd.cut(df_clean.carat, bins=10)

## Dela upp datasetet i tre: original, korrekt carat baserad på x, y, z och inkorrekt carat

In [9]:
#- För rund: ((x + y)/2)² × z × 0.0061 med 2 % felmarginal
#- För övriga: x × y × z × 0.0062
diamonds_with_correct_carat = []
diamonds_with_incorrect_carat =[]
round_square_diamonds = []
error_margin = 0.01

for index, row in df_clean.iterrows():
    x = row['x']
    y = row['y']
    z = row['z']
    actual_carat = row['carat']
    
    if abs(x - y) <= 0.02 * ((x + y)/2):
        round_square_diamonds.append(index)
        recalculated_carat = ((x + y)/2)**2 * z * 0.0061
        
        relative_error = abs(recalculated_carat - actual_carat) /actual_carat
        
        if relative_error <= error_margin:
            diamonds_with_correct_carat.append(index)
        else:
            diamonds_with_incorrect_carat.append(index)
    else:
        recalculated_carat = x * y * z * 0.0062
        
        relative_error = abs(recalculated_carat - actual_carat) /actual_carat
        
        if relative_error <= error_margin:
            diamonds_with_correct_carat.append(index)
        else:
            diamonds_with_incorrect_carat.append(index)
            
print(f"Diamanter med korrekt beräknad carat: {len(diamonds_with_correct_carat)}")
print(f"Diamanter med inkorrekt beräknad carat: {len(diamonds_with_incorrect_carat)}")
print(f"Diamanter som antingen är runda eller kvadratiska: {len(round_square_diamonds)}")



Diamanter med korrekt beräknad carat: 23375
Diamanter med inkorrekt beräknad carat: 29955
Diamanter som antingen är runda eller kvadratiska: 52623


In [10]:
# Sparar om datan
df_clean_corr_carat = df_clean.loc[diamonds_with_correct_carat].copy()
df_clean_inc_carat = df_clean.loc[diamonds_with_incorrect_carat].copy()
df_clean_original = df_clean.copy()


## Ta bort x, y, z


In [11]:
df_clean_original_without_x_y_z = df_clean_original.drop(['x', 'y', 'z'], axis=1)
df_clean_corr_carat_without_x_y_z = df_clean_corr_carat.drop(['x', 'y', 'z'], axis=1)
df_clean_inc_carat_without_x_y_z = df_clean_inc_carat.drop(['x', 'y', 'z'], axis=1)


In [12]:
df_clean_corr_carat_without_x_y_z.to_csv('../data/diamonds_clean_cor_without_x_y_z.csv', index=True)

In [13]:
print(df_clean_original_without_x_y_z)
df_clean_original_without_x_y_z.to_csv('../data/diamonds_clean_without_x_y_z.csv', index=True)



       carat  depth  table  price  cut_encoded  color_encoded  clarity_encoded
0       0.23   61.5   55.0    326            4              5                1
1       0.21   59.8   61.0    326            3              5                2
2       0.23   56.9   65.0    327            1              5                4
3       0.29   62.4   58.0    334            3              1                3
4       0.31   63.3   58.0    335            1              0                1
...      ...    ...    ...    ...          ...            ...              ...
53935   0.72   60.8   57.0   2757            4              6                2
53936   0.72   63.1   55.0   2757            1              6                2
53937   0.70   62.8   60.0   2757            2              6                2
53938   0.86   61.0   58.0   2757            3              2                1
53939   0.75   62.2   55.0   2757            4              6                1

[53330 rows x 7 columns]


# Normalisera datan

In [14]:
numeric_cols = ['carat', 'depth', 'table', 'price', 'cut_encoded', 'color_encoded', 'clarity_encoded']
scaler = StandardScaler()

df_clean_corr_carat_without_x_y_z[numeric_cols] = scaler.fit_transform(df_clean_corr_carat_without_x_y_z[numeric_cols])
df_clean_inc_carat_without_x_y_z[numeric_cols] = scaler.fit_transform(df_clean_inc_carat_without_x_y_z[numeric_cols])
df_clean_original_without_x_y_z[numeric_cols] = scaler.fit_transform(df_clean_original_without_x_y_z[numeric_cols])




In [15]:
# Sparar dataset till nya csv-filer
df_clean_original_without_x_y_z.to_csv('../data/normalized_diamonds_clean_without_x_y_z.csv', index=True)
df_clean_corr_carat_without_x_y_z.to_csv('../data/normalized_diamonds_clean_corr_carat_without_x_y_z.csv', index=True)
df_clean_inc_carat_without_x_y_z.to_csv('../data/normalized_diamonds_clean_inc_carat_without_x_y_z.csv', index=True)
